# 다양한 framework에서의 테스트

In [1]:
import os; os.chdir("../")

In [2]:
# import torch
# import pickle
# import functools
# import pandas as pd
# import numpy as np
# import xgboost as xgb

# from ucimlrepo import fetch_ucirepo
# from sklearn.model_selection import train_test_split

# Framework

# AutoXAI

In [3]:
import sys; sys.path.append("lib/AutoXAI")

In [4]:
from sklearn.datasets import load_diabetes, load_wine
import pandas as pd

# 데이터 불러오기
wine = load_wine()

# pandas DataFrame으로 변환
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.Series(wine.target, name='target') > 6

# 데이터 확인
print(X.head())
print(y.head())

data = pd.concat([X, y], axis=1)
data.to_csv("wine.csv", index=False)


from sklearn.neural_network import MLPClassifier
import pickle
# MLPRegressor 모델 생성
model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
# 모델 학습
model.fit(X, y)
# 모델 저장
with open("mlp_diabetes_model.pkl", "wb") as f:
    pickle.dump(model, f)

X = X.values
y = y.values
feature_names = wine.feature_names

   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  
0                  

In [5]:
from hyperparameters_optimization import get_parameters, gp_optimization
from XAI_solutions import set_up_explainer, get_local_exp


context = {}
context['model'] = model

question = "Why"
verbose = False
task = "classification"
session_id = 'session-name'
scaling = 'Std'
weights = [1.0, 2.0, 0.5]
distance = "cosine"

context["X"] = X
context["y"] = y
context["feature_names"] = feature_names
context["verbose"] = verbose
context["task"] = task
context["question"] = question
context["session_id"] = session_id
context["scaling"] = scaling
context["weights"] = weights
context["distance"] = distance
context["explanations"] = []
evstrat_list = ["IS","ES"]

if evstrat_list != None:
    context["ES"] = True if 'ES' in evstrat_list else False
    context["IS"] = True if 'IS' in evstrat_list else False
else:
    context["ES"] = False
    context["IS"] = False

score_hist = {}
score_hist["xai_sol"] = []
score_hist["epoch"] = []
score_hist["aggregated_score"] = []
score_hist["parameters"] = []

properties_list = ['robustness', 'fidelity', 'conciseness']
xai_list = ["LIME", "SHAP"]


# TODO to data class property
for property in properties_list:
    score_hist[property] = []
    score_hist["scaled_"+property] = []

for xai_sol in xai_list:
    print("XAI solution:", xai_sol)
    score_hist["xai_sol"].append(xai_sol)
    score_hist["epoch"].append(-1)

    parameters = get_parameters(
        xai_sol, score_hist, "default", properties_list, context)
    score_hist["parameters"].append(parameters)

    n = len(context["X"])
    X = context["X"]
    exp_values = []
    context['explainer'] = set_up_explainer(xai_sol, parameters, context)
    for i in range(n):
        exp_values += get_local_exp(xai_sol, X[i], parameters, context)


/home/cwl/miniconda3/envs/autoxai/lib/python3.8/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/cwl/miniconda3/envs/autoxai/lib/python3.8/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimizat

XAI solution: LIME


Using 178 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


XAI solution: SHAP


In [6]:
# python launch.py data/diabetes.csv "target" regression -m models/mlp_diabetes_model.pkl -q "Why" -p "robustness,fidelity,conciseness" --hpo "gp" --evstrat "ES,IS" -w "1,2,0.5" --scaling "Std" -s 0 -e 25

NameError: name 'STOP' is not defined

In [ ]:
get_local_exp(xai_sol, X[i], parameters, context)

KeyError: 0

## OpenXAI

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from openxai.dataloader import ReturnLoaders
from openxai import LoadModel
from openxai import Explainer
from openxai.experiment_utils import fill_param_dict
from openxai.model import ArtificialNeuralNetwork

In [ ]:
# explainers_dict = {
#     'grad': Gradient,
#     'sg': SmoothGrad,
#     'itg': InputTimesGradient,
#     'ig': IntegratedGradients,
#     'shap': SHAPExplainerC,
#     'lime': LIME,
#     'control': RandomBaseline
# }

# loader_train, loader_test = ReturnLoaders(data_name='german', download=True)
# # get an input instance from the test dataset
# inputs, labels = next(iter(loader_test))

# model = LoadModel(data_name= 'german', ml_model='ann', pretrained=True)
# X_train = torch.tensor([d[0] for d in loader_train.dataset])

# param_dict = fill_param_dict('lime', {}, X_train)
# exp_method = Explainer(method='lime', model=model, param_dict=param_dict)
# explanations= exp_method.get_explanations(inputs, label=labels)

In [ ]:
dataset_nm = "Adult"
X_train = np.load(f"data/{dataset_nm}/X_train.npy")
y_train = np.load(f"data/{dataset_nm}/y_train.npy")

X_test = np.load(f"data/{dataset_nm}/X_test.npy")
y_test = np.load(f"data/{dataset_nm}/y_test.npy")

feature_metadata = pickle.load(open(f"data/{dataset_nm}/feature_metadata.pkl", "rb"))

model = ArtificialNeuralNetwork(input_dim=X_test.shape[1], hidden_layers=[100,100])

In [ ]:
train_data = torch.tensor(X_train, dtype=torch.float32)
input_data = torch.tensor(X_test, dtype=torch.float32)

In [ ]:
# param_dict = fill_param_dict('lime', {}, train_data)
# exp_method = Explainer(method='lime', model=model, param_dict=param_dict)

# exp_method = Explainer(method='grad', model=model)
# exp_method = Explainer(method='sg', model=model)
# exp_method = Explainer(method='itg', model=model)

# param_dict = fill_param_dict('ig', {}, train_data)
# exp_method = Explainer(method='ig', model=model, param_dict=param_dict)


exp_method = Explainer(method='shap', model=model)




In [ ]:
y_pred = model(input_data).detach().argmax(dim=1)
total_exp = []
dataset = TensorDataset(input_data, y_pred)
loader = DataLoader(dataset, batch_size=10)
for inputs, labels in loader:
    explanations= exp_method.get_explanations(inputs, label=labels)
    total_exp.append(explanations)
    break

explanations = torch.cat(total_exp, dim=0)
explanations.shape

torch.Size([10, 111])

In [ ]:
feature_metadata

_exp = []
for nm, info in feature_metadata.items():
    if info['type'] == 'categorical':
        idx = info['index']
        onehot = input_data[:10, idx]
        val = explanations[:, idx]
        cat_exp = (onehot * val).sum(dim=1)
        _exp.append(cat_exp)
    else:
        _exp.append(explanations[:, info['index']])

exp = torch.stack(_exp, dim=1)
exp.shape

torch.Size([10, 14])

In [ ]:
exp

tensor([[ 3.7067e-03, -6.2093e-03, -1.7211e-03, -6.9594e-04,  3.8262e-04,
          1.6946e-04, -6.6975e-04,  5.7893e-04, -8.1775e-04, -8.0424e-04,
          4.6599e-04,  2.4125e-04,  1.9976e-04, -3.9700e-03],
        [-4.3620e-03,  3.2987e-03,  1.5954e-03, -1.5827e-03,  1.3526e-03,
         -1.8663e-03, -7.3954e-04, -2.4865e-03, -1.1802e-03,  1.9820e-03,
         -3.5694e-04, -3.7238e-05,  5.9745e-04, -2.3326e-03],
        [ 7.2948e-03,  1.5306e-04, -3.1804e-05,  8.6214e-05, -4.7309e-05,
         -4.4448e-04,  1.8585e-03,  2.8437e-03, -3.4996e-04,  1.4028e-03,
         -3.5091e-04,  2.0247e-04,  9.6643e-05, -2.3110e-03],
        [ 8.5133e-04, -6.5700e-03, -1.1147e-03, -1.2950e-03, -7.8500e-05,
          2.2973e-03, -5.1395e-04,  4.6578e-03, -1.3155e-03, -1.7015e-03,
          1.0156e-04,  3.6321e-04, -1.1688e-04, -3.4267e-03],
        [-4.9224e-04,  1.3795e-03,  8.7882e-04, -8.4938e-04,  1.4198e-04,
          8.6840e-04,  1.8120e-03,  3.8192e-03,  1.7027e-03,  9.9984e-04,
         -7.

In [ ]:
STOP

In [ ]:
type(model), type(inputs), type(labels), type(explanations), inputs.shape, labels.shape, labels.dtype, type(loader_test)

(openxai.model.ArtificialNeuralNetwork,
 torch.Tensor,
 torch.Tensor,
 torch.Tensor,
 torch.Size([32, 60]),
 torch.Size([32]),
 torch.int64,
 torch.utils.data.dataloader.DataLoader)

In [ ]:
inputs[0]

tensor([0.5909, 0.1146, 1.0000, 0.3333, 0.2000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000], dtype=torch.float64)

## OmniXAI

In [ ]:
from omnixai.data.tabular import Tabular
from omnixai.preprocessing.tabular import TabularTransform
from omnixai.explainers.tabular import TabularExplainer

2025-03-13 14:48:57.608476: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-13 14:48:59.146516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741844939.815529   30461 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741844939.977498   30461 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 14:49:01.687092: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
dataset_nm = "Adult"
X_train = np.load(f"data/{dataset_nm}/X_train.npy")
y_train = np.load(f"data/{dataset_nm}/y_train.npy")

X_test = np.load(f"data/{dataset_nm}/X_test.npy")
y_test = np.load(f"data/{dataset_nm}/y_test.npy")

feature_metadata = pickle.load(open(f"data/{dataset_nm}/feature_metadata.pkl", "rb"))
raw_data = pd.read_csv(f"data/{dataset_nm}/raw_data.csv")


xgb_model = xgb.XGBClassifier()
xgb_model.load_model(f"data/{dataset_nm}/xgb_model.json")

nn_model = torch.nn.Sequential(
    torch.nn.Linear(X_train.shape[1], 3),
    torch.nn.Softmax(dim=1)
)

train_data = Tabular(raw_data, categorical_columns=[c for c in raw_data.columns if feature_metadata[c]["type"] == "categorical"])

In [ ]:
def _transform(X, feature_metadata):
    input_data = []
    for k, v in feature_metadata.items():
        preprocessed = v['encoder'].transform(X[[k]].values)
        if v['type'] == 'categorical':
            preprocessed = preprocessed.toarray()
        input_data.append(preprocessed)
    
    input_array = np.concatenate(input_data, axis=1)
    return input_array

def _invert_input_array(input_array, feature_metadata):
    inverted_data = {}
    
    for col, meta in feature_metadata.items():
        if meta['type'] == 'categorical':
            # One-hot encoded 된 부분 추출
            start_idx, end_idx = meta['index'][0], meta['index'][-1] + 1
            cat_data = input_array[:, start_idx:end_idx]
            # OneHotEncoder로 복원
            inverted_col = meta['encoder'].inverse_transform(cat_data)
            inverted_data[col] = inverted_col.flatten()
        else:
            # 수치형 데이터 복원
            idx = meta['index']
            num_data = input_array[:, idx].reshape(-1, 1)
            inverted_col = meta['encoder'].inverse_transform(num_data)
            inverted_data[col] = inverted_col.flatten()
    
    # 복원된 데이터를 DataFrame으로 변환
    inverted_df = pd.DataFrame(inverted_data)
    
    return inverted_df
        
transform = functools.partial(_transform, feature_metadata=feature_metadata)
invert_input_array = functools.partial(_invert_input_array, feature_metadata=feature_metadata)

In [ ]:
tmp = transform(raw_data)

In [ ]:
explainer = TabularExplainer(
  explainers=["LimeTabular", "ShapTabular", "lime", "shap"], # The explainers to apply
  mode="classification",                             # The task type
  data=train_data,
  model=xgb_model.predict_proba,                                     # The ML model to explain
  preprocess=lambda z: transform(z.data)
)

nn_explainer = TabularExplainer(
  # explainers=["LimeTabular", "ShapTabular", "ig"], # The explainers to apply
  explainers=["LimeTabular", "ShapTabular"], # The explainers to apply
  mode="classification",                             # The task type
  data=train_data,
  model=nn_model,                                       # The ML model to explain
  preprocess=lambda z: torch.tensor(transform(z.data), dtype=torch.float32)
)

In [ ]:
test_instances = invert_input_array(X_test[:10])

explanations = explainer.explain(test_instances)
nn_explanations = nn_explainer.explain(test_instances)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
idx = 4
# explanations['LimeTabular'].get_explanations(idx)['features'], explanations['LimeTabular'].get_explanations(idx)['scores']
# explanations['ShapTabular'].get_explanations(idx)['features'], explanations['ShapTabular'].get_explanations(idx)['scores']
explanations['lime'].get_explanations(idx)['features'], explanations['lime'].get_explanations(idx)['scores']
# explanations['shap'].get_explanations(idx)['features'], explanations['shap'].get_explanations(idx)['scores']

(['capital-gain',
  'marital-status',
  'capital-loss',
  'hours-per-week',
  'occupation',
  'age',
  'sex',
  'native-country',
  'fnlwgt',
  'education-num'],
 [-0.6916381338688814,
  0.12350883830059717,
  -0.07013317629306902,
  0.05951879371883744,
  0.05440253382722002,
  0.04146769210204393,
  0.025463251237053054,
  0.014590300825607144,
  0.012842448468914323,
  0.01055183746359225])

In [ ]:
explanations['LimeTabular'].get_explanations(1)['features'], explanations['LimeTabular'].get_explanations(1)['scores']

(['capital-gain',
  'capital-loss',
  'hours-per-week',
  'age',
  'marital-status',
  'native-country',
  'sex',
  'fnlwgt',
  'race',
  'occupation'],
 [0.7146692980670353,
  0.09948085944391453,
  0.09032331528379359,
  -0.059015724128280496,
  0.05270170754617013,
  -0.029825946975223127,
  -0.019937145021878502,
  0.019450162959702547,
  -0.016697319733481562,
  0.01373394956749946])

In [ ]:
explanations['LimeTabular'].get_explanations(0).keys()

dict_keys(['instance', 'features', 'values', 'scores', 'target_label'])

In [ ]:
feature_metadata

{'age': {'type': 'numerical', 'encoder': StandardScaler(), 'index': 0},
 'workclass': {'type': 'categorical',
  'encoder': OneHotEncoder(),
  'cat_dist': array([3.75905983e-02, 2.93190287e-02, 6.42070349e-02, 2.04741821e-04,
         6.94197617e-01, 3.47037386e-02, 7.90712911e-02, 4.05593547e-02,
         4.29957823e-04, 1.97166373e-02]),
  'index': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
 'fnlwgt': {'type': 'numerical', 'encoder': StandardScaler(), 'index': 11},
 'education': {'type': 'categorical',
  'encoder': OneHotEncoder(),
  'cat_dist': array([0.02843864, 0.03709922, 0.01345154, 0.00505712, 0.01042136,
         0.01955284, 0.01547848, 0.03277917, 0.04219729, 0.16430531,
         0.01216166, 0.32316449, 0.0543999 , 0.00169936, 0.01707547,
         0.22271815]),
  'index': array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])},
 'education-num': {'type': 'numerical',
  'encoder': StandardScaler(),
  'index': 28},
 'marital-status': {'type': 'categorical

In [ ]:
raw_data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

/home/cwl/miniconda3/envs/pnpenv/lib/python3.10/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [ ]:
# Load the dataset
feature_names = [
   "Age", "Workclass", "fnlwgt", "Education",
   "Education-Num", "Marital Status", "Occupation",
   "Relationship", "Race", "Sex", "Capital Gain",
   "Capital Loss", "Hours per week", "Country", "label"
]
df = pd.DataFrame(
  np.genfromtxt('adult.data', delimiter=', ', dtype=str),
  columns=feature_names
)
tabular_data = Tabular(
   df,
   categorical_columns=[feature_names[i] for i in [1, 3, 5, 6, 7, 8, 9, 13]],
   target_column='label'
)

In [ ]:
transformer = TabularTransform().fit(tabular_data)
class_names = transformer.class_names
x = transformer.transform(tabular_data)

df.shape, x.data.shape

((32561, 15), (32561, 109))

In [ ]:
from omnixai.preprocessing.tabular import TabularTransform
# Data preprocessing
transformer = TabularTransform().fit(tabular_data)
class_names = transformer.class_names
x = transformer.transform(tabular_data)
# Split into training and test datasets
train, test, train_labels, test_labels = \
    train_test_split(x[:, :-1], x[:, -1], train_size=0.80)
# Train an XGBoost model (the last column of `x` is the label column after transformation)
model = xgboost.XGBClassifier(n_estimators=300, max_depth=5)
model.fit(train, train_labels)
# Convert the transformed data back to Tabular instances
train_data = transformer.invert(train)
test_data = transformer.invert(test)

In [ ]:
train, train.shape

(array([[0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         6.000e+01],
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         5.000e+01],
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         4.000e+01],
        ...,
        [1.000e+00, 0.000e+00, 0.000e+00, ..., 7.688e+03, 0.000e+00,
         5.000e+01],
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         5.000e+01],
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         4.000e+01]]),
 (26048, 108))

In [ ]:
# OmniXAI는 LIME, SHAP, TreeSHAP, IG를 갖추고 있다.
from omnixai.explainers.tabular import TabularExplainer
# Initialize a TabularExplainer
explainer = TabularExplainer(
  explainers=["lime", "shap", "mace", "pdp", "ale", "aaa"], # The explainers to apply
  mode="classification",                             # The task type
  data=train_data,                                   # The data for initializing the explainers
  model=model,                                       # The ML model to explain
  preprocess=lambda z: transformer.transform(z),     # Converts raw features into the model inputs
  params={
     "mace": {"ignored_features": ["Sex", "Race", "Relationship", "Capital Loss"]}
  }                                                  # Additional parameters
)




AssertionError: Explainer aaa is not found. Please choose explainers from dict_keys(['LimeTabular', 'ShapTabular', 'PartialDependenceTabular', 'ALE', 'SensitivityAnalysisTabular', 'L2XTabular', 'PermutationImportance', 'GlobalShapTabular', 'BiasAnalyzer', 'MACEExplainer', 'GPTExplainer', 'CounterfactualExplainer', 'KNNCounterfactualExplainer', 'IntegratedGradientTabular', 'LinearRegression', 'LogisticRegression', 'TreeRegressor', 'TreeClassifier', 'ShapTreeTabular', 'lime', 'shap', 'pdp', 'partial_dependence', 'ale', 'accumulated_local_effects', 'sa', 'sensitivity', 'l2x', 'L2X', 'permutation', 'shap_global', 'bias', 'mace', 'gpt', 'ce', 'counterfactual', 'ce_knn', 'knn_ce', 'ig', 'integrated_gradient', 'linear_regression', 'logistic_regression', 'tree_regressor', 'tree_classifier', 'shap_tree'])

In [ ]:
# Generate explanations
test_instances = test_data[:5]
local_explanations = explainer.explain(X=test_instances)



local_explanations
# global_explanations = explainer.explain_global(
#     params={"pdp": {"features": ["Age", "Education-Num", "Capital Gain",
#                                  "Capital Loss", "Hours per week", "Education",
#                                  "Marital Status", "Occupation"]}}
# )

  0%|          | 0/5 [00:00<?, ?it/s]

OrderedDict([('predict',
              <omnixai.explanations.base.PredictedResults at 0x7f949ee72a10>),
             ('lime',
              [{'instance':     Age Workclass    fnlwgt     Education  Education-Num Marital Status  \
              0  20.0   Private  110998.0  Some-college           10.0  Never-married   
              
                   Occupation Relationship                Race     Sex  Capital Gain  \
              0  Adm-clerical    Own-child  Asian-Pac-Islander  Female           0.0   
              
                 Capital Loss  Hours per week        Country  
              0           0.0            30.0  United-States  , 'features': ['Capital Gain', 'Age', 'Marital Status', 'Hours per week', 'Capital Loss', 'Sex', 'fnlwgt', 'Occupation', 'Relationship', 'Race'], 'values': [0.0, 20.0, 'Never-married', 30.0, 0.0, 'Female', 110998.0, 'Adm-clerical', 'Own-child', 'Asian-Pac-Islander'], 'scores': [0.7098656386832946, 0.0986917834142603, 0.09079057400844752, 0.086275902

In [ ]:
local_explanations['lime'].get_explanations(1)

{'instance':     Age         Workclass    fnlwgt  Education  Education-Num  \
 1  48.0  Self-emp-not-inc  160724.0  Bachelors           13.0   
 
        Marital Status       Occupation Relationship                Race   Sex  \
 1  Married-civ-spouse  Exec-managerial      Husband  Asian-Pac-Islander  Male   
 
    Capital Gain  Capital Loss  Hours per week Country  
 1           0.0           0.0            40.0   Japan  ,
 'features': ['Capital Gain',
  'Marital Status',
  'Hours per week',
  'Education-Num',
  'Age',
  'Occupation',
  'Country',
  'Capital Loss',
  'Sex',
  'Race'],
 'values': [0.0,
  'Married-civ-spouse',
  40.0,
  13.0,
  48.0,
  'Exec-managerial',
  'Japan',
  0.0,
  'Male',
  'Asian-Pac-Islander'],
 'scores': [-0.7174269202573026,
  0.12373506933540021,
  -0.08009449263879101,
  0.07463333708307905,
  0.07408962211553939,
  0.07013468779567734,
  -0.0497145992611175,
  -0.042112685273002576,
  0.030125166776901604,
  0.009174592742435584],
 'target_label': 1}

In [ ]:
import numpy as np


In [ ]:
X_test = np.load("data/Wine Quality/X_test.npy")
y_test = np.load("data/Wine Quality/y_test.npy")

input_data = np.concatenate((X_test, y_test.reshape(-1, 1)), axis=1)
tabular_data = Tabular(
   input_data,
   categorical_columns=[],
   target_column=input_data.shape[1] - 1
)

In [ ]:
nn_model = torch.nn.Linear(X_test.shape[1], 2)

transformer = TabularTransform().fit(tabular_data)
class_names = transformer.class_names
x = transformer.transform(tabular_data)
# Split into training and test datasets
train, test, train_labels, test_labels = \
    train_test_split(x[:, :-1], x[:, -1], train_size=0.80)
# Train an XGBoost model (the last column of `x` is the label column after transformation)
# Convert the transformed data back to Tabular instances
train_data = transformer.invert(torch.tensor(train))
test_data = transformer.invert(torch.tensor(test))

explainer = TabularExplainer(
  explainers=["lime", "shap", "ig"], # The explainers to apply
  mode="classification",                             # The task type
  data=torch.randn(100,2),
  model=nn_model,                                       # The ML model to explain
  preprocess=lambda z: transformer.transform(z),     # Converts raw features into the model inputs
)




AssertionError: Explainer lime -- training_data should be an instance of Tabular.

## OpenXAI

In [ ]:
import torch
from openxai.dataloader import ReturnLoaders
loader_train, loader_test = ReturnLoaders(data_name='german', download=True)
# get an input instance from the test dataset
inputs, labels = next(iter(loader_test))

In [ ]:
from openxai import LoadModel
model = LoadModel(data_name= 'german', ml_model='ann', pretrained=True)

In [ ]:
X_train = torch.tensor([d[0] for d in loader_train.dataset])

In [ ]:
from openxai import Explainer
from openxai.experiment_utils import fill_param_dict
param_dict = fill_param_dict('lime', {}, X_train)
exp_method = Explainer(method='lime', model=model, param_dict=param_dict)
explanations= exp_method.get_explanations(inputs, label=labels)

In [ ]:
inputs.shape

torch.Size([32, 60])

In [ ]:
explanations.shape

torch.Size([32, 60])

In [ ]:
from openxai import Evaluator
from openxai.explainers.perturbation_methods import NormalPerturbation
import numpy as np

feature_metadata = np.where((inputs.numpy() == 0.0) | (inputs.numpy() == 1.0), 'd', 'c')[0].tolist()

metric_evaluator = Evaluator(model, metric='PGI')
kwargs = {
    "explanations": explanations,
    "inputs": inputs,
    "k": 0.1,
    "perturb_method": NormalPerturbation('tabular'),
    "feature_metadata": feature_metadata
}

score = metric_evaluator.evaluate(**kwargs)

In [ ]:
score

(array([0.61782783, 0.20031753, 0.17158128, 0.32372916, 0.4453343 ,
        0.00382134, 0.1159284 , 0.01427623, 0.41874745, 0.15988487,
        0.00148061, 0.52968276, 0.19574705, 0.08563896, 0.26961112,
        0.19825181, 0.39603913, 0.26493934, 0.10648002, 0.48599267,
        0.01620395, 0.16462477, 0.14538735, 0.3685585 , 0.29140532,
        0.49489924, 0.01769307, 0.07826665, 0.35052273, 0.4042034 ,
        0.1675385 , 0.06326002], dtype=float32),
 0.2364961)

In [ ]:
METRICS = ["AbPC"]
FRAMEWORK = ["PnPXAI", "OmniXAI", "OpenXAI"]
MODEL = ["LogisticRegression", "XGBoost", "TabResNet", "TabTransformer"]

In [ ]:
import torch
model = torch.nn.Linear(10, 2)

In [ ]:
from pnpxai.evaluator.metrics.pixel_flipping import AbPC
metric_obj = AbPC(model)
inputs = {"input" : torch.randn(20, 10)}
explanations = torch.randn(20, 10)
labels = torch.randint(0, 2, (20,))
evals = metric_obj.evaluate(inputs, labels, explanations)

In [ ]:
evals, evals.shape

(tensor([-0.0181,  0.0345,  0.0346,  0.0213,  0.0264, -0.1871,  0.0195, -0.1951,
         -0.1298,  0.0007, -0.1259,  0.0353, -0.0921,  0.0799,  0.0821, -0.0270,
         -0.0424, -0.0208, -0.1354, -0.2065]),
 torch.Size([20]))